In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

from preprocess import get_cifar100_datasets

train_ds, val_ds, test_ds = get_cifar100_datasets(image_size=(128, 128), batch_size=64)

In [10]:
def vgg19(input_shape=(128, 128, 3), num_classes=100):
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Block 5
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2, 2)))

    # Fully Connected Layers
    model.add(Flatten())
    # model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu', kernel_regularizer=l2(1e-4)))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax')) 

    return model


In [11]:
model_VGG19 = vgg19()
model_VGG19.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_VGG19.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization_32 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
activation_32 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 128, 128, 64)      36928     
_________________________________________________________________
batch_normalization_33 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
activation_33 (Activation)   (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 64, 64)       

In [12]:
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

early_stop = EarlyStopping(patience=5, restore_best_weights=True)
history = model_VGG19.fit(
    train_ds,
    epochs=4,
    validation_data=val_ds,
    callbacks=[early_stop, lr_schedule],
)

Epoch 1/4
704/704 [==============================] - 215s 304ms/step - loss: 4.7884 - accuracy: 0.0086 - val_loss: 4.6605 - val_accuracy: 0.0100
Epoch 2/4
704/704 [==============================] - 217s 308ms/step - loss: 4.6461 - accuracy: 0.0081 - val_loss: 4.6333 - val_accuracy: 0.0100
Epoch 3/4
284/704 [===========>..................] - ETA: 2:07 - loss: 4.6307 - accuracy: 0.0088

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model_VGG19.evaluate(test_ds, verbose=1)
print(f'Test accuracy: {test_accuracy:.4f}')